In [1]:
from carbon.helpers.stdimports import *

plt.style.use('seaborn-dark')
plt.rcParams['figure.figsize'] = [12,6]
require_version("2.1")

[stdimports] Carbon v2.3-BETA0 (21/Jan/2022)
[stdimports] CarbonSimulatorUI v2.4 (12/Jan/2023)
[stdimports] CarbonOrderUI v1.6.1 (21/Jan/2023)


# Expanding Ranges

Set up a simulation with the following strategy

- 10,000 USD to buy ETH at 1,500
- Sell range set to 2,000-3,000 (initially empty and zero capacity)

In [2]:
Sim = CarbonSimulatorUI(pair="ETH/USDC")
r = Sim.add_strategy("USDC", 15000, 1500, 1500, 0, 2000, 3000)
r["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,15000.0,15000.0,USDC,False,1500.0,1500.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,ETH,0.0,0.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,0


Looking at the above we see that the SELL ETH curve has zero capacity. It will therefore be filled _across the range_ as we see in the trade below.

We now sell 1500 USDC, netting us 1 ETH

In [3]:
r = Sim.amm_sells("USDC", 1500*5)
r["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,0,0,A,AMM sells 7500USDC buys 5ETH,True,True,True,None,7500.0,USDC,5.0,ETH,ETHUSDC,[0],1,1500.0,USDC per ETH


In [4]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,15000.0,7500.0,USDC,False,1500.0,1500.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,ETH,5.0,5.0,ETH,False,2000.0,3000.0,2000.0,USDC per ETH,0


We see above that we now have 1 ETH on the curve; this 1 ETH. This ETH is evenly distruted over the curve (look at `price` in subsequent sells)

In [5]:
Sim.amm_sells("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,1,1,A,AMM sells 1ETH buys 2076USDC,True,True,True,None,1.0,ETH,2076.197882,USDC,ETHUSDC,[1],1,2076.197882,USDC per ETH


In [6]:
Sim.amm_sells("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,2,2,A,AMM sells 1ETH buys 2241USDC,True,True,True,None,1.0,ETH,2240.665817,USDC,ETHUSDC,[1],1,2240.665817,USDC per ETH


In [7]:
Sim.amm_sells("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,3,3,A,AMM sells 1ETH buys 2425USDC,True,True,True,None,1.0,ETH,2425.482442,USDC,ETHUSDC,[1],1,2425.482442,USDC per ETH


In [8]:
Sim.amm_sells("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,4,4,A,AMM sells 1ETH buys 2634USDC,True,True,True,None,1.0,ETH,2634.148991,USDC,ETHUSDC,[1],1,2634.148991,USDC per ETH


In [9]:
Sim.amm_sells("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,5,5,A,AMM sells 1ETH buys 2871USDC,True,True,True,None,1.0,ETH,2870.953581,USDC,ETHUSDC,[1],1,2870.953581,USDC per ETH


Note the ETH curve is now empty (marginal price = 3000 is at top end)

In [10]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,19747.448714,19747.448714,USDC,False,1500.0,1500.0,1500.0,USDC per ETH,1
1,1,ETHUSDC,ETH,5.000000,0.000000,ETH,False,2000.0,3000.0,3000.0,USDC per ETH,0


Now we buy some more ETH (ie we sell USDC). Note the curve is now already expanded to 5ETH

In [11]:
Sim.amm_buys("ETH", 1)["trades"].query("aggr == True")

,uid,id,subid,note,aggr,exec,partial,limitfail,amt1,tkn1,amt2,tkn2,pair,routeix,nroutes,price,p_unit
0,6,6,A,AMM sells 1500USDC buys 1ETH,True,True,False,None,1500.0,USDC,1.0,ETH,ETHUSDC,[0],1,1500.0,USDC per ETH


We see that the marginal price moved down (to 2747)

In [12]:
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,19747.448714,18247.448714,USDC,False,1500.0,1500.0,1500.000000,USDC per ETH,1
1,1,ETHUSDC,ETH,5.000000,1.000000,ETH,False,2000.0,3000.0,2747.458155,USDC per ETH,0


And the marginal price moves down again

In [13]:
Sim.amm_buys("ETH", 1)["trades"].query("aggr == True")
Sim.state()["orders"]

,id,pair,tkn,y_int,y,y_unit,disabled,p_start,p_end,p_marg,p_unit,lid
0,0,ETHUSDC,USDC,19747.448714,16747.448714,USDC,False,1500.0,1500.0,1500.000000,USDC per ETH,1
1,1,ETHUSDC,ETH,5.000000,2.000000,ETH,False,2000.0,3000.0,2525.512861,USDC per ETH,0
